# Imports

In [1]:
#torch: package containing multi-dim tensor data structure
import torch
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data as data

# torchvision: 
# package that provides access to popular datasets, model architectures, and image transformations for computer vision
import torchvision 
import torchvision.transforms as transforms
import torchvision.models as models

from PIL import Image
import requests
import io

torch.set_printoptions(linewidth=120)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on the GPU')
else:
    device = torch.device('cpua') 
    print('running on the CPU')

running on the GPU


# Define Model Parameters

In [3]:
EPOCHS = 30
BATCH_SIZE = 25
LEARNING_RATE = 0.001

TRAIN_DATA_PATH = './images/train/'
TEST_DATA_PATH = './images/test/'

TRANSFORM_IMG = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                   ])

# Load Data

In [4]:
train_set = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, 
                                             transform=TRANSFORM_IMG)

train_loader = data.DataLoader(train_set, 
                               batch_size=BATCH_SIZE, 
                               shuffle=True)

test_set = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, 
                                            transform=TRANSFORM_IMG)

test_loader = data.DataLoader(test_set, 
                              batch_size=BATCH_SIZE, 
                              shuffle=True)

In [5]:
print(test_set.class_to_idx)

{'hot_dog': 0, 'not_hot_dog': 1}


In [6]:
label_map = {0: 'hotdog',
             1: 'not hotdog'}

In [7]:
model = torch.hub.load('pytorch/vision', 'mobilenet_v2', pretrained=True).to(device)

Using cache found in C:\Users\tsail/.cache\torch\hub\pytorch_vision_master


In [8]:
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=False),
    torch.nn.Linear(in_features=1280, out_features=2),
).to(device)


In [9]:
best_accuracy = 0.0
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    model.train()
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_set))
    print('EPOCH %d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'model.pt')
        best_accuracy = test_accuracy

EPOCH 0: 0.676707
EPOCH 1: 0.757028
EPOCH 2: 0.795181
EPOCH 3: 0.827309
EPOCH 4: 0.845382
EPOCH 5: 0.867470
EPOCH 6: 0.853414
EPOCH 7: 0.855422
EPOCH 8: 0.873494
EPOCH 9: 0.879518
EPOCH 10: 0.899598
EPOCH 11: 0.889558
EPOCH 12: 0.907631
EPOCH 13: 0.907631
EPOCH 14: 0.895582
EPOCH 15: 0.907631
EPOCH 16: 0.893574
EPOCH 17: 0.915663
EPOCH 18: 0.907631
EPOCH 19: 0.917671
EPOCH 20: 0.923695
EPOCH 21: 0.917671
EPOCH 22: 0.933735
EPOCH 23: 0.943775
EPOCH 24: 0.927711
EPOCH 25: 0.915663
EPOCH 26: 0.937751
EPOCH 27: 0.931727
EPOCH 28: 0.935743
EPOCH 29: 0.941767


In [10]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test images: {} %'.format(100 * correct / total))
    print('Correct: {}'.format(correct))
    print('Total: {}'.format(total))

Accuracy of the network on the test images: 92.36947791164658 %
Correct: 460
Total: 498
